# Complexidade de letras:

## Introdução:

Este código foi desenvolvido com o intuito de explorar e revelar a complexidade das letras musicais, proporcionando uma análise detalhada de sua riqueza lírica. Diversos critérios foram considerados para avaliar a profundidade e sofisticação das composições, incluindo a diversidade temática das letras. A seguir, estão os principais critérios utilizados na análise:

Número de sílabas: Palavras com mais sílabas geralmente indicam maior complexidade, tanto em termos de compreensão quanto de pronúncia.

Frequência de palavras: A frequência com que as palavras aparecem na letra pode sugerir sua acessibilidade e familiaridade para o público.

Análise gramatical: Frases com muitos gerúndios tendem a ser mais coloquiais, enquanto aquelas com maior uso de conjunções subordinadas indicam um tom mais formal.

Diversidade lexical: A variedade de vocabulário e a riqueza semântica contribuem significativamente para a expressividade e profundidade das letras.

Diversidade temática: A amplitude dos temas abordados nas letras reflete sua complexidade, com composições que exploram uma gama mais ampla de assuntos sendo consideradas mais ricas e elaboradas.

Tamanho da letra

Tamanho dos versos

##  2 - Desenvolvimento:

### 2.1 - Importando bibliotecas:

In [68]:
import pandas as pd
import textstat as tst
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

### 2.2 - Leitura do dataset:

In [69]:
df = pd.read_csv("datasets/dataset_atualizado.csv")

### 2.3 - Descrição do dataset utilizado:

Artist: Artista

Track_name: Nome da música

Lyrics: Letra da música

genre: Gênero musical

In [70]:
df.head()

,artist,track_name,lyrics,genre
0,abba,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",pop
1,abba,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...",pop
2,abba,As Good As New,I'll never know why I had to go \r\nWhy I had...,pop
3,abba,Bang,Making somebody happy is a question of give an...,pop
4,abba,Bang-A-Boomerang,Making somebody happy is a question of give an...,pop


### 2.4 - Limpeza de dados:

Como são letras de música, as linhas contêm \r ou \n, o que, futuramente, pode ocasionar problemas de entendimento, sendo necessário filtrá-las.

In [71]:
def limpar_texto(texto):
    texto = texto.replace("\r\n", " ").replace("\n", " ").replace("<br/>", "\n").replace("<p>", " ").replace("</p>", "\n").strip()
    return texto

Criar uma nova coluna com as letras filtradas:

In [72]:
df['letras_limpas'] = df['lyrics'].apply(limpar_texto)

In [73]:
df.head()

,artist,track_name,lyrics,genre,letras_limpas
0,abba,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",pop,"Look at her face, it's a wonderful face And ..."
1,abba,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...",pop,"Take it easy with me, please Touch me gently..."
2,abba,As Good As New,I'll never know why I had to go \r\nWhy I had...,pop,I'll never know why I had to go Why I had to...
3,abba,Bang,Making somebody happy is a question of give an...,pop,Making somebody happy is a question of give an...
4,abba,Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,Making somebody happy is a question of give an...


Indexar os nomes das músicas para evitar que músicas de nomes iguais, mas de contextos diferentes sejam confundidas:

In [75]:
df['track_name'] = [f"{index} - {name}" for index, name in enumerate(df['track_name'])]

In [76]:
df.head()

,artist,track_name,lyrics,genre,letras_limpas
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",pop,"Look at her face, it's a wonderful face And ..."
1,abba,"1 - Andante, Andante","Take it easy with me, please \r\nTouch me gen...",pop,"Take it easy with me, please Touch me gently..."
2,abba,2 - As Good As New,I'll never know why I had to go \r\nWhy I had...,pop,I'll never know why I had to go Why I had to...
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,Making somebody happy is a question of give an...
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,Making somebody happy is a question of give an...


## 3 - Análises exploratórias:

Nessa parte do projeto, focaremos em procurar ferramentas que facilitem nossa classificação de complexidade das letras musicais

### 3.1 - Número de sílabas:

Este código tem como objetivo contar o número de sílabas de cada palavra em letras de músicas e identificar palavras que possuem uma certa complexidade lírica. Para isso, utilizaremos uma biblioteca chamada `textstat`, que tem uma função chamada `syllable_count`, a qual calcula a quantidade de sílabas nas palavras

### Passo 1: Criar um Dicionário de Sílaba

Primeiro, criamos um dicionário que armazena o número de sílabas de cada palavra em cada música. Usaremos dictionary comprehension, em que o nome da música será a key e a quantidade de sílabas de cada palavra será o Value. 

In [80]:
dic_silabas = {
    track: {palavra: tst.syllable_count(palavra) for palavra in letras.split(" ")} 
    for track, letras in zip(df['track_name'], df['letras_limpas']) 
}

### Passo 2: Contar Palavras Complexas

Aqui, percorremos as músicas e suas palavras para identificar palavras que possuem 5 ou mais sílabas. Também filtramos expressões repetitivas para uma análise mais precisa.

In [81]:
# Dicionário para armazenar a contagem de palavras com 4 ou mais sílabas em cada música
musicas_silabas_complexas = {}

for nome_musica, palavras in dic_silabas.items():
    for palavra, silabas in palavras.items():
        # Verifica se a palavra tem 4 ou mais sílabas
        if silabas >= 5:
            # Filtra palavras que contêm uma letra quatro ou mais vezes (ex.: "Ahhh"), comum em músicas
            if not any(palavra.count(letra) >= 4 for letra in set(palavra)):
                if nome_musica not in musicas_silabas_complexas:
                    musicas_silabas_complexas[nome_musica] = 1
                else:
                    musicas_silabas_complexas[nome_musica] += 1

### Passo 3: Criar uma coluna no dataset

Aqui, criaremos uma coluna referente a quantidade de sílabas maiores que 4 que a letra possui

In [82]:
df['Silabas'] = df['track_name'].map(musicas_silabas_complexas).fillna(0)

In [85]:
df['Silabas']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
45541    0.0
45542    0.0
45543    0.0
45544    0.0
45545    0.0
Name: Silabas, Length: 45546, dtype: float64

### 3.2 - Frequência:

Este código visa calcular a diversidade lexical das letras das músicas, utilizando a medida TTR (Type-Token Ratio), que é a razão entre o número de palavras únicas e o total de palavras em uma letra. 

### Passo 1: Calcular o TTR (Type-Token Ratio)

Aqui, calculamos o TTR para cada letra, dividindo o número de palavras únicas pelo total de palavras na letra. O resultado é armazenado em uma lista para análise posterior.

In [87]:
lista_ttr = [ len(set(df.letras_limpas[letra].lower().split())) / len(df.letras_limpas[letra].split())
    if len(df.letras_limpas[letra].split()) > 0 else 0
    for letra in range(len(df))
]

### Passo 2: Criar uma coluna no dataset

In [88]:
df["Frequencia"] = lista_ttr

In [89]:
df["Frequencia"]

0        0.457516
1        0.307692
2        0.362179
3        0.440000
4        0.454545
           ...   
45541    0.481481
45542    0.381625
45543    0.450199
45544    0.542857
45545    0.291883
Name: Frequencia, Length: 45546, dtype: float64

### 3.3 - Análise Gramatical:

Este código tem como objetivo calcular uma pontuação com base nas tags gramaticais das palavras presentes nas letras das músicas. A pontuação é atribuída de acordo com a presença de substantivos, adjetivos, advérbios, verbos e outras categorias gramaticais. Para realizar a análise gramatical, utilizamos a biblioteca NLTK (Natural Language Toolkit), que fornece ferramentas para a tokenização e a análise de partes do discurso (POS tagging).

### Passo 1: Inicializar o Dicionário de Pontuação

Primeiramente, criamos um dicionário (`dic_tags`) onde cada música é uma chave e sua pontuação inicial é zero. Isso nos permitirá acumular a pontuação ao longo da análise.

In [90]:
dic_tags = {track: 0 for track in df['track_name']} 

### Passo 2: Iterar Sobre as Letras e Calcular a Pontuação

Neste passo, utilizamos um loop para percorrer cada letra do DataFrame. Para cada letra, realizamos a tokenização e a análise de partes do discurso (POS tagging) para identificar a categoria gramatical de cada palavra. Em seguida, atribuímos uma pontuação com base nas tags gramaticais.

In [91]:
for index, row in df.iterrows():
    tokens = word_tokenize(row['letras_limpas'].lower())  # Tokeniza a letra em palavras
    tags = nltk.pos_tag(tokens) 
    
    pontuacao = 0  
    for palavra, tag in tags:
        # Atribui pontuação com base nas tags gramaticais
        if tag in ['NNP', 'NNPS']:  # Substantivos próprios
            pontuacao += 0.02
        elif tag in ['JJ', 'JJR', 'JJS']:  # Adjetivos
            pontuacao += 0.01
        elif tag in ['RB', 'RBR', 'RBS']:  # Advérbios
            pontuacao += 0.01
        elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:  # Verbos
            pontuacao += 0.01
        elif tag == 'WDT':  # Determinante interrogativo
            pontuacao += 0.01 
        elif tag == 'IN':  # Preposição ou conjunção subordinativa 
            pontuacao += 0.01
            
    dic_tags[row['track_name']] = pontuacao 

### Passo 3: Criar uma coluna no dataset

In [92]:
df['Pontuacao Gramatical'] = df['track_name'].map(dic_tags)

In [110]:
df['Pontuacao Gramatical']

0        0.71
1        1.32
2        1.74
3        1.03
4        1.06
         ... 
45541    0.85
45542    1.07
45543    1.02
45544    0.73
45545    2.69
Name: Pontuacao Gramatical, Length: 45546, dtype: float64

### 3.4 - Diversidade Lexical:

Este trecho de código tem como objetivo calcular a complexidade das letras das músicas usando o modelo Word2Vec. A complexidade é determinada pela variância dos vetores das palavras, permitindo uma análise da riqueza e diversidade vocabular presente nas letras. A variância é utilizada porque indica o grau de dispersão dos vetores em relação à média, refletindo a diversidade vocabular: uma maior variância sugere um vocabulário mais rico e variado, enquanto uma menor variância indica similaridade entre as palavras, resultando em uma letra potencialmente menos complexa.

### Passo 1: Remoção de Palavras Comuns (Stop Words)

Primeiro, criamos um conjunto de palavras comuns (stop words) que serão removidas das letras, uma vez que essas palavras geralmente não contribuem para a variabilidade temática.

In [93]:
stop_words = set(stopwords.words('english')) 

### Passo 2: Preparar lista de palavras para o modelo

O modelo requer que as palavras estejam em listas separadas para facilitar a análise, visto que ele calcula as palavras pela coocorrência

In [94]:
letras = [[letra for letra in df['letras_limpas'][n].split() if letra not in stop_words] for n in range(len(df))]

### Passo 3: Treinamento do Modelo Word2Vec

Em seguida, utilizamos o Word2Vec para criar um modelo que representa as palavras em um espaço vetorial. Isso nos permitirá calcular a complexidade das letras com base nos vetores gerados.

In [95]:
model = Word2Vec(letras, vector_size=100, window=5, min_count=1, workers=4)  

### Passo 4: Definição das Funções

#### Passo 4.1 - Cálculo da Variância dos Vetores

A função `calcular_variancia_vetores` tem como objetivo calcular a soma das variâncias dos vetores correspondentes a uma lista de palavras, utilizando Word2Vec.

- Passos:
  1. Para cada palavra na lista, o código verifica se ela está presente no vocabulário do modelo.
  2. Se a palavra estiver no modelo, o vetor correspondente à palavra é adicionado a uma lista.
  3. Após coletar os vetores, o código os normaliza utilizando `MinMaxScaler`.
  4. Em seguida, calcula a variância ao longo do eixo 0 (variância de cada dimensão dos vetores).
  5. Retorna a soma dessas variâncias.

In [96]:
def calcular_variancia_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:  # Verifica se a palavra está no vocabulário do modelo
            vetores.append(modelo.wv[palavra])  
    
    if len(vetores) > 0:  # Evitar calcular variância em lista vazia
        vetores_np = np.array(vetores)  # Converte a lista de vetores em um array NumPy

        scaler = MinMaxScaler()
        vetores_normalizados = scaler.fit_transform(vetores_np)
        
        variancia = np.var(vetores_normalizados, axis=0)  # Calcula a variância ao longo do eixo 0
        
        return np.sum(variancia)  
        
    else:
        return 0 


#### Passo 4.2 - Cálculo da Complexidade da Letra

A função calcular_complexidade processa a letra da música e calcula a sua complexidade utilizando a função complexidade_vetores.

In [97]:
def avaliar_complexidade_letra(letra, modelo):
    # Remove stopwords e processa a letra
    letra_processada = [palavra.lower() for palavra in letra.split() if palavra not in stop_words]
    return calcular_variancia_vetores(letra_processada, modelo)

### Passo 5 - Cálculo da Complexidade para Cada Música

Aqui, inicializamos um dicionário (dic_complexidade) para armazenar a complexidade de cada música e iteramos sobre as letras para calcular essa complexidade.

In [98]:
dic_complexidade = {df.track_name[n]: {} for n in range(len(df))}  

for index, row in df.iterrows():
    letra = row['letras_limpas']
    complexidade = avaliar_complexidade_letra(letra, model)  
    dic_complexidade[df.track_name[index]] = complexidade  

lista_div = [dic_complexidade[df.track_name[n]] for n in range(len(df))]

In [99]:
df['diversidade lexical'] = lista_div

In [111]:
df['diversidade lexical']

0        5.178668
1        4.460471
2        4.138115
3        3.592463
4        3.550966
           ...   
45541    4.828613
45542    3.965744
45543    3.578850
45544    4.332500
45545    3.232071
Name: diversidade lexical, Length: 45546, dtype: float32

### 3.5 - Diversidade temática

Este código tem como objetivo analisar a presença de temas nas letras das músicas, utilizando um modelo Word2Vec para calcular a similaridade entre os vetores das palavras da letra e os vetores de temas predefinidos. A seguir, descrevemos as principais etapas do processo:

### Passo 1: Definição de Temas e Palavras-chave

In [100]:
themes = {
    'love': ['love', 'affection', 'heart', 'passion', 'desire', 'romance'],
    'sadness': ['sadness', 'pain', 'longing', 'loneliness', 'lament', 'loss'],
    'freedom': ['freedom', 'live', 'free', 'independence', 'choice', 'escape'],
    'friendship': ['friendship', 'companion', 'loyalty', 'trust', 'camaraderie'],
    'hope': ['hope', 'faith', 'dream', 'future', 'light', 'renewal'],
    'nature': ['nature', 'earth', 'sky', 'sea', 'forest', 'wildlife'],
    'struggle': ['struggle', 'battle', 'strength', 'resistance', 'challenge', 'conquest'],
    'joy': ['joy', 'happiness', 'laughter', 'smile', 'celebration', 'fun'],
    'solitude': ['solitude', 'isolation', 'distance', 'disconnection', 'emptiness'],
    'nostalgia': ['nostalgia', 'memories', 'past', 'remembrance', 'reminiscence'],
    'change': ['change', 'transformation', 'growth', 'evolution', 'new beginnings'],
    'self-love': ['self-love', 'self-esteem', 'self-awareness', 'acceptance', 'confidence'],
    'conflict': ['conflict', 'dispute', 'disagreement', 'tension', 'polarization'],
    'solidarity': ['solidarity', 'help', 'support', 'community', 'union'],
    'socio-political': ['socio-political', 'justice', 'rights', 'freedom of speech', 'equality'],
    'party': ['party', 'celebration', 'fun', 'dance', 'joy'],
    'mental health': ['mental health', 'anxiety', 'depression', 'healing', 'balance'],
}

### Passo 2: Cálculo da Média dos Vetores de Palavras

A função media_vetores recebe uma lista de palavras e o modelo Word2Vec, retornando a média dos vetores dessas palavras. Isso permite representar um conjunto de palavras como um único vetor, que captura a essência do significado coletivo.

In [101]:
def media_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:
            vetores.append(modelo.wv[palavra])
    
    if len(vetores) == 0:
        return np.zeros(modelo.vector_size)  # Retorna um vetor de zeros se não houver palavras válidas
    
    return np.mean(vetores, axis=0)

### Passo 3: Normalizar

Para evitar grandes diferenças entre datasets de tamanhos diferentes

In [102]:
def normalizar_similaridade(similaridade, vetor_letra):
    # Normaliza pela magnitude do vetor letra
    magnitude_letra = np.linalg.norm(vetor_letra)
    
    if magnitude_letra != 0:
        return similaridade / magnitude_letra
    return similaridade

### Passo 4: Cálculo da Similaridade Temática

A função resultado é responsável por comparar a letra da música (representada por seu vetor) com os vetores de cada tema. Para cada tema, a função:

a) Calcula o vetor médio das palavras-chave do tema.

Motivação: O vetor médio representa a essência do tema, capturando a semântica das palavras associadas a ele. Isso permite que a análise considere o contexto das palavras, facilitando a comparação com a letra.

b) Compara a similaridade (usando a similaridade cosseno) entre o vetor da letra e o vetor do tema.

Motivação: A similaridade cosseno mede a proximidade entre dois vetores, indicando quão semelhantes são os significados representados. Essa comparação ajuda a identificar a relação temática entre a letra da música e os temas predefinidos.

In [103]:
def resultado(themes, model, vetor_letra):
    resultado = {}
    for tema, palavras_chave in themes.items():
        vetor_tema = media_vetores(palavras_chave, model)

        if vetor_tema is not None and vetor_letra is not None and np.any(vetor_tema) and np.any(vetor_letra):
            similaridade = model.wv.cosine_similarities(vetor_letra, np.array([vetor_tema]))
            
            # Aplica a normalização da similaridade
            similaridade_normalizada = normalizar_similaridade(similaridade[0], vetor_letra)
            resultado[tema] = similaridade_normalizada
        else:
            resultado[tema] = 0
    
    return resultado

### Passo 5: Iteração Sobre as Letras das Músicas

O código itera sobre cada linha do DataFrame df, processando as letras das músicas:

a) Converte as letras para minúsculas e remove palavras de parada (stop words).

b) Calcula o vetor da letra usando a função media_vetores.

c) Chama a função resultado para obter a similaridade entre a letra e os temas, armazenando os resultados em um dicionário chamado dic_div_tematica.

In [104]:
# Cria um dicionário vazio para armazenar a diversidade temática de cada música, usando o nome da música como chave.
dic_div_tematica = {df.track_name[n]: {} for n in range(len(df))}

for index, row in df.iterrows():
    letra = row['letras_limpas'] 
    
    # Processa a letra, convertendo todas as palavras para minúsculas e removendo palavras de parada.
    letra_processada = [palavra.lower() for palavra in letra.split() if palavra.lower() not in stop_words]
    
    vetor_letra = media_vetores(letra_processada, model)
    
    # Obtem o resultado da similaridade temática entre a letra e os temas definidos.
    res = resultado(themes, model, vetor_letra)
    
    dic_div_tematica[df.track_name[index]] = res

In [ ]:
lista_desvios = [np.std(list(value.values())) for value in dic_div_tematica.values()]
df["diversidade tematica"] = lista_desvios

In [112]:
df["diversidade tematica"]

0        0.024275
1        0.023414
2        0.026448
3        0.030630
4        0.030784
           ...   
45541    0.027352
45542    0.021152
45543    0.025230
45544    0.025217
45545    0.024529
Name: diversidade tematica, Length: 45546, dtype: float32

### 3.6 - Tamanho das letras

In [105]:
dic_tamanho = {row['track_name']: len(row['letras_limpas'].split()) for index, row in df.iterrows()}
df['Tamanho'] = df['track_name'].map(dic_tamanho)

### 3.7 - Repetição de versos:

In [106]:
def pontuacao_verso(dic_contagem_versos):
    dic = {}
    for key, value in dic_contagem_versos.items():
        total_versos = sum(value.values())  # Total de ocorrências de versos
        versos_unicos = len(value)  # Número de versos únicos
        
        # Calcula o comprimento médio dos versos
        comprimento_medio_versos = total_versos / versos_unicos if versos_unicos > 0 else 0
        dic[key] = comprimento_medio_versos
    
    return dic


In [107]:
dic_contagem_versos = {df.track_name[n]: Counter() for n in range(len(df))} 

# Itera sobre as linhas do DataFrame para contar os versos
for index, row in df.iterrows():
    versos = row['lyrics'].split('\n')  # Divide a letra em versos
    contagem_versos = Counter(verso.strip().lower() for verso in versos if verso.strip())
    
    # Atualiza a contagem de versos para a música atual
    dic_contagem_versos[row['track_name']].update(contagem_versos)


In [108]:
resultados = pontuacao_verso(dic_contagem_versos)

In [109]:
df['Versos'] = df['track_name'].map(resultados)

In [114]:
df.head()

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica,Tamanho,Versos
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",pop,"Look at her face, it's a wonderful face And ...",0.0,0.457516,0.71,5.178668,0.024275,153,1.333333
1,abba,"1 - Andante, Andante","Take it easy with me, please \r\nTouch me gen...",pop,"Take it easy with me, please Touch me gently...",0.0,0.307692,1.32,4.460471,0.023414,260,2.136364
2,abba,2 - As Good As New,I'll never know why I had to go \r\nWhy I had...,pop,I'll never know why I had to go Why I had to...,0.0,0.362179,1.74,4.138115,0.026448,312,1.434783
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,Making somebody happy is a question of give an...,0.0,0.440000,1.03,3.592463,0.030630,200,1.307692
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,Making somebody happy is a question of give an...,0.0,0.454545,1.06,3.550966,0.030784,198,1.307692


## Classificação: *

In [ ]:
# Carregar o dataset original (já treinado)
df = pd.read_csv('complexo.csv')
X = df[['Silabas', 'Frequencia', 'Pontuacao Gramatical', 'diversidade lexical', 'diversidade tematica',  'Tamanho', 'Versos']]
y = df['Complexidade']  # Assumindo que esta é a coluna com as classes

# Treinar o modelo
model = RandomForestClassifier()
model.fit(X, y)

# Carregar o novo dataset
novo_dataset = pd.read_csv('dataset_teste.csv')
X_novo = novo_dataset[['Silabas', 'Frequencia', 'Pontuacao Gramatical', 'diversidade lexical', 'diversidade tematica',  'Tamanho', 'Versos']]

# Fazer previsões
y_pred_novo = model.predict(X_novo)
novo_dataset['Predicao'] = y_pred_novo

# Salvar o novo dataset com previsões
novo_dataset.to_csv('dataset_teste.csv', index=False)